### Jingyi Wang uni: jw3562
### Siwen Tang uni: st3101

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
import keras
import pandas as pd

Using TensorFlow backend.
/Users/Selina/anaconda/envs/python3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


## Load Data and Preprocess

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,stratify =iris.target )

In [3]:
num_classes=3
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)
num_classes = y_train.shape[1]

## Gridsearch CV for hidden layer and regularization

In [8]:
%%capture
# with regularization
 
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from keras import regularizers

def make_model(optimizer="adam",hidden_size=64,regular=0.01):
    model = Sequential()
    model.add(Dense(hidden_size,input_shape=(4,),kernel_regularizer = regularizers.l2(regular),activity_regularizer = regularizers.l1(regular)))
    model.add(Activation('relu'))

    model.add(Dense(hidden_size,kernel_regularizer = regularizers.l2(regular),activity_regularizer = regularizers.l1(regular)))
    model.add(Activation('relu'))

    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(optimizer = optimizer, loss="categorical_crossentropy",metrics = ['accuracy'])


    model.compile(optimizer = optimizer, loss="categorical_crossentropy",metrics = ['accuracy'])
    return model
clf = KerasClassifier(make_model)
param_grid = {'hidden_size':[32,64],
              'epochs':[64,256],
             'regular':[0.0001,0.001,0.1]
             }
grid = GridSearchCV(clf,param_grid=param_grid,cv=5)
grid.fit(X_train, y_train)

In [9]:
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))

best mean cross-validation score: 0.973
best parameters: {'regular': 0.0001, 'hidden_size': 64, 'epochs': 256}
38/38 [==============================] - 1s 31ms/step
test-set score: 0.974


In [10]:
res = pd.DataFrame(grid.cv_results_)
res.pivot_table(index=["param_epochs", "param_hidden_size","param_regular"],
                values=['mean_train_score', "mean_test_score"])

/Users/Selina/anaconda/envs/python3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Selina/anaconda/envs/python3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/Selina/anaconda/envs/python3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kw

mean_test_score  \
param_epochs param_hidden_size param_regular                    
64           32                0.0001                0.955357   
                               0.0010                0.964286   
                               0.1000                0.392857   
             64                0.0001                0.955357   
                               0.0010                0.964286   
                               0.1000                0.348214   
256          32                0.0001                0.964286   
                               0.0010                0.964286   
                               0.1000                0.276786   
             64                0.0001                0.973214   
                               0.0010                0.964286   
                               0.1000                0.276786   

                                              mean_train_score  
param_epochs param_hidden_size param_regular                    
64           32                0.0001                 0.979900  
                               0.0010                 0.964270  
                               0.1000                 0.459875  
             64                0.0001                 0.979900  
                               0.0010                 0.970961  
                               0.1000                 0.442272  
256          32                0.0001                 1.000000  
                               0.0010                 0.995531  
                               0.1000                 0.368365  
             64                0.0001                 1.000000  
                               0.0010                 1.000000  
                               0.1000                 0.350412

### test score is 0.974